In [ ]:
import json
import urllib
import pandas as pd 
import numpy as np
from io import StringIO 
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import collections
import heapq
import geopy
from geopy.geocoders import Nominatim, GoogleV3
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json, string
import requests
import geocoder
from folium import Map, Marker, GeoJson, LayerControl
from ediblepickle import checkpoint
from tqdm import tqdm_notebook
import os 
import folium
from folium import plugins
%matplotlib inline

In [ ]:
coordinates = open('coordinates.co').readlines()#ile containing node coordinates. Each line follows this structure: (Id_Node, Latitude, Longitude)
distance = open('distancegraph.gr').readlines() #file containing physical distances between each pair of nodes. Each line follows this structure: (Id_Node1, Id_Node2, d(Id_Node1,Id_Node2)), where d(x,y) is the physical distance between x and y.
traveltime = open('traveltimegraph.gr').readlines() #file containing time distances between each pair of nodes. Each line follows this structure: (Node1, Node2, t(Id_Node1, Id_Node2)), where t(x,y) is the time distance between x and y.
coordinates = [item.replace("\n", "") for item in coordinates]
distance = [item.replace("\n", "") for item in distance]
traveltime = [item.replace("\n", "") for item in traveltime]
coordinates = coordinates[7:]
distance = distance[7:]
traveltime = traveltime[7:]
coordinates = pd.DataFrame([map(int,raw.strip("\n").strip('v ').split()) for raw in coordinates],columns = ['Id_Node', 'Latitude', 'Longitude'])
traveltime= pd.DataFrame([map(int,raw.strip("\n").strip('a ').split()) for raw in traveltime],columns = ['Node1', 'Node2', 'TimeTravel'])
distance = pd.DataFrame([map(int,raw.strip("\n").strip('a ').split()) for raw in distance],columns = ['Node1', 'Node2', 'Distance'])

Built a dictionary : node as key {connection node, distance, time distance, latitude, longitude)

In [ ]:
from collections import defaultdict
d = defaultdict(list)
for index in tqdm_notebook(distance.index, desc='Fill the Dictionary'):
    d[distance["Node1"].iloc[index]].append((distance["Node2"].iloc[index],distance["Distance"].iloc[index],traveltime["TimeTravel"].iloc[index],1))
#np.save('my_file.npy', d)

### Functionality 2 - Find the smartest Network!
### It takes in input: a set of nodes v = {v_1, ..., v_n}
One of the following distances function: t(x,y), d(x,y) or network distance (i.e. consider all edges to have weight equal to 1).
Implement an algorithm that returns the set of roads (edges) that enable the user to visit all the places. We want this set to be the ones whose sum of distances is minimum.

As a dummy example, a set of input could be {Colosseo, Piazza Venezia, Piazza del Popolo} and therefore the associated set of streets will be {Via dei Fori Imperiali, Via del Corso}.


#### It means find -> minimum spanning tree ->kruskal
#### we can consider as our weights the time or the distance 

pseudocode:
1. Sort all the edges in non-decreasing order of their weight
2. Pick the smallest edge. Check if it forms a cycle with the spanning tree formed so far. If cycle is not formed, include this edge. Else, discard it.
3. Repeat step #2 untile there are (V-1) edges in the spanning tree

NETWORKX
0

You are solving the Steiner tree problem, which is NP-hard. networkx provides an approximation algorithm to solve this problem: networkx.algorithms.approximation.steinertree.steiner_tree.

https://www.geeksforgeeks.org/steiner-tree/
https://gis.stackexchange.com/questions/307336/generating-steiner-tree-using-network-x-in-python

FIRST TRY OF SPANNING TREE

In [200]:
def spanning_tree_distance(a):
    print('Choose from which node start')
    node=int(input())
    mst = defaultdict(set)
    visited = set([node])
    edges = [(dist,node,to) for to,dist,_,_ in d[node]]
    heapq.heapify(edges)

    while edges:
        dist, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next, dist,_,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (dist, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst,node


In [201]:
def spanning_tree_time(d):
    print('Choose from which node start')
    node=int(input())   
    mst = defaultdict(set)
    visited = set([node])
    edges = [(time,node,to) for to,_,time,_ in d[node]]
    heapq.heapify(edges)

    while edges:
        time, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next,_,time,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (time, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst



In [202]:

def spanning_tree_weight(d):
    print('Choose from which node start')
    node=int(input())
    mst = defaultdict(set)
    visited = set([node])
    edges = [(weight,node,to) for to,_,_,weight in d[node]]
    heapq.heapify(edges)

    while edges:
        weight, frm, to = heapq.heappop(edges) #Pop and return the smallest item from the heap, maintaining the heap invariant.
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next,_,time,_ in d[to]:
                if to_next not in visited:
                    heapq.heappush(edges, (weight, to, to_next)) #Push the value item into the heap, maintaining the heap invariant.
    return mst


Display Numerical Result

In [206]:
from IPython.display import display, HTML
print('Choose which method use for find the smartest Network:Time, Weight, Distance. p.s. If you are not interest just write "quit"') 
#while a!='quit':
a=input().lower()
if a=='time':
    b=dict(spanning_tree_time(d))
    display(b)

elif a=='weight':
    display(spanning_tree_weight(d))
    b=dict(b)



elif a=='distance':
    display(spanning_tree_distance(d))
    b=dict(b)

elif a=='quit':
    print(':(')

else:
    print('What you said? please repeat!')





Choose which method use for find the smartest Network:Time, Weight, Distance. p.s. If you are not interest just write "quit"
time
Choose from which node start
60


{60: {37, 1048640},
 1048640: {134},
 37: {38, 1048607},
 1048607: {1048608},
 1048608: {39},
 39: {1048624},
 1048624: {33},
 33: {1048598},
 1048598: {28, 1048603},
 28: {2650, 1050699},
 1050699: {1050700},
 1050700: {2652, 1048601},
 1048601: {31, 1050322},
 31: {1050698},
 1050698: {2654, 1048627},
 1048627: {64},
 64: {2655, 1050705},
 2655: {2656, 1050706},
 2656: {2657, 1050707},
 2657: {2658, 1048626},
 1048626: {63, 179, 2176},
 179: {180},
 180: {35, 2664},
 35: {2665, 1048604},
 2665: {2666, 1050714, 1050716},
 1050714: {2667, 2668, 1050715},
 1048604: {1048606, 1050321},
 1048606: {2662},
 2662: {1050711, 1050717},
 1050717: {2661, 2669},
 2661: {40, 1050718},
 40: {1048609, 1050710},
 1048609: {2663, 1050708},
 1050708: {43, 1050709},
 43: {2660, 1048612},
 1048612: {2384},
 2384: {1048594, 1050496},
 1050496: {1050241},
 1050241: {1050239},
 1048594: {23, 2659},
 2663: {41, 1050713},
 41: {1048610, 1050712},
 2664: {36},
 36: {1048605},
 1050239: {2074, 1050238},
 2654: 

    Visualization 2 - Visualize the smartest Network!
    Visualization of the map 
    
 

In [218]:
result_spanning=pd.DataFrame(list(b.items()), columns=['Id_Node', 'Connection'])

In [254]:
df_final = pd.merge(result_spanning, coordinates, on='Id_Node', how='inner')

In [256]:
df_final=df_final[:1000]

In [257]:
#fist longitude and then latitude, the resultino['longitude, latitude'] are divided by one milion because coordinates are something like 45.0003
map = folium.Map(location=[np.median((df_final['Longitude']/1000000).tolist()),
                           np.median((df_final['Latitude']/1000000).tolist())], default_zoom_start=15)#keep the median to localize our data

for i in tqdm_notebook(range(len(df_final))):#
    folium.features.RegularPolygonMarker(location = [((df_final['Longitude']/1000000).values)[i], ((df_final['Latitude']/1000000).values)[i]],
                                             number_of_sides = 3,
                                             radius = 10,
                                             weight = 4,
                                             fill_opacity = 0.8).add_to(map)

#to add the edge between two nodes
for i in tqdm_notebook(range(len(df_final)-1)):
    folium.PolyLine(locations = [(((df_final['Longitude']/1000000).values)[i], ((df_final['Latitude']/1000000).values)[i]), (((df_final['Longitude']/1000000).values)[i+1], ((df_final['Latitude']/1000000).values)[i+1])], 
                    line_opacity = 0.5).add_to(map) 
#weight represent the dimension of the node
map.save("Spanning_tree.html")
#this take two hours and half to execute the first loop ( of all the spanning tree!!)
#takes 5 hours to execute the second 
map

'reference'
https://deparkes.co.uk/2019/02/27/folium-lines-and-markers/